In [1]:
import os
cwd =  os.getcwd().replace("notebooks","")
os.chdir(cwd)

In [2]:
# --- Robust notebook shim for legacy joblib artifacts expecting `encoders.*` ---
import sys, types, numpy as np

# Create/replace a lightweight 'encoders' module in sys.modules
enc_mod = types.ModuleType("encoders")

try:
    from sentence_transformers import SentenceTransformer
except Exception as e:
    SentenceTransformer = None
    print("NOTE: sentence-transformers not available:", e)

class _SBERTBase:
    """
    Compat shim implementing the sklearn Transformer API expected by saved Pipelines.
    Handles pickles that don't call __init__ and are missing attributes.
    Provides both class names: SBERTEncoder and SBERTFeaturizer.
    """
    # NOTE: __init__ might not be called during unpickle; use _ensure_attrs() everywhere.
    def __init__(self, model="sentence-transformers/all-MiniLM-L6-v2", **kwargs):
        self.model_name = model
        self._enc = None
        self._kwargs = kwargs

    def _ensure_attrs(self):
        # Add any attributes that might be missing from legacy pickles
        if not hasattr(self, "model_name") or self.model_name is None:
            self.model_name = "sentence-transformers/all-MiniLM-L6-v2"
        if not hasattr(self, "_enc"):
            self._enc = None
        if not hasattr(self, "_kwargs"):
            self._kwargs = {}

    def _ensure_encoder(self):
        self._ensure_attrs()
        if self._enc is None:
            if SentenceTransformer is None:
                raise RuntimeError(
                    "sentence-transformers not installed in this kernel; "
                    "pip install sentence-transformers && restart kernel"
                )
            self._enc = SentenceTransformer(self.model_name)

    # sklearn API
    def fit(self, X, y=None):
        self._ensure_attrs()
        return self

    def transform(self, X):
        self._ensure_encoder()
        return np.asarray(self._enc.encode(list(X), show_progress_bar=False))

    # some older code may call .encode directly; alias it
    def encode(self, X):
        return self.transform(X)

# Expose both legacy names on the encoders module
class SBERTEncoder(_SBERTBase): ...
class SBERTFeaturizer(_SBERTBase): ...

enc_mod.SBERTEncoder = SBERTEncoder
enc_mod.SBERTFeaturizer = SBERTFeaturizer
sys.modules["encoders"] = enc_mod

# Make sure your package code is importable too (if needed)
import pathlib
if str(pathlib.Path("src").resolve()) not in sys.path:
    sys.path.append(str(pathlib.Path("src").resolve()))
print("encoders shim ready (SBERTEncoder + SBERTFeaturizer) and sys.path configured")

encoders shim ready (SBERTEncoder + SBERTFeaturizer) and sys.path configured


In [3]:
import joblib
from pathlib import Path

def load_mapper():
    for name in [".artifacts/defi_mapper.joblib", ".artifacts/defi_mapper_embed.joblib"]:
        p = Path(name).resolve()
        if p.exists():
            print("Loading:", p.as_posix())
            return joblib.load(p.as_posix())
    raise FileNotFoundError("No mapper artifact found in .artifacts/")

pipe = load_mapper()
print(pipe)


Loading: /Users/ian_moore/repos/micro-lm/.artifacts/defi_mapper.joblib
Pipeline(steps=[('sbertencoder', <__main__.SBERTEncoder object at 0x3090ff340>),
                ('calibratedclassifiercv',
                 CalibratedClassifierCV(cv=3,
                                        estimator=LogisticRegression(C=8.0,
                                                                     class_weight='balanced',
                                                                     max_iter=2000,
                                                                     random_state=0),
                                        method='isotonic'))])


In [4]:
import os, sys, warnings, importlib, __main__, json
warnings.filterwarnings("ignore", category=FutureWarning)

# Path to your workspace (src + scripts)
ROOT = os.getcwd()
SRC = os.path.join(ROOT, "src")
SCRIPTS = os.path.join(ROOT, "scripts")
for p in (SRC, SCRIPTS):
    if p not in sys.path: sys.path.insert(0, p)

# Legacy pickle compat for mappers (if needed)
def legacy_install_inline(target="micro_lm.domains.defi.benches.encoders"):
    mod = importlib.import_module(target)
    sys.modules.setdefault("encoders", mod)
    for name in ("SBERTEncoder","SbertEncoder","EmbedVectorizer","SBERTVectorizer"):
        if hasattr(mod, name) and not hasattr(__main__, name):
            setattr(__main__, name, getattr(mod, name))
legacy_install_inline()

# Use the harness's runner (same path the CLI uses)
from scripts.tier2_benchmark import run_once

def tier2_run_micro(domain, prompt, *, context, policy, rails="stage11", T=1):
    """Notebook-friendly Tier-2 call that returns plan/verify/flags."""
    out = run_once(domain=domain, prompt=prompt, context=context, policy=policy, rails=rails, T=T)
    # normalize to a compact dict
    return {
        "plan": out.get("plan"),
        "verify": out.get("verify"),
        "flags": out.get("flags"),
        "raw": out,  # keep original if you want to inspect
    }

# Example call
policy = {
    "ltv_max": 0.75, "hf_min": 1.0,
    "audit": {"backend": "wdd", "mode": "pure", "profile": False, "max_null": 64, "batch": 32},
    "mapper": {"model_path": ".artifacts/defi_mapper.joblib", "confidence_threshold": 0.45},
}
context = {"oracle": {"age_sec": 5, "max_age_sec": 30}}

res = tier2_run_micro("defi", "supply 7.0245 SOL to maker", context=context, policy=policy)
print(json.dumps({"verify":res["verify"], "plan":res["plan"], "flags":res["flags"]}, indent=2))


[debug] mapper fallback for 'supply 7.0245 SOL to maker' -> deposit_asset
{
  "verify": {
    "ok": false,
    "reason": "abstain_non_exec"
  },
  "plan": {},
  "flags": {
    "mapper_fallback": true
  }
}


In [5]:
# one-time in your environment (outside the notebook):
# pip uninstall -y micro-lm
# pip install -e .   # from your repo root

# then in the notebook, before importing:
import os, sys
sys.path.insert(0, os.path.join(os.getcwd(), "src"))
for m in list(sys.modules):
    if m == "micro_lm" or m.startswith("micro_lm."):
        del sys.modules[m]
from micro_lm.core.runner import run_micro  # now the real rails-backed version


In [9]:
# --- Tier-2 rails in a notebook (DeFi, end-to-end, no shim) ---
import os, sys, warnings, importlib, __main__, json
warnings.filterwarnings("ignore", category=FutureWarning)

ROOT = os.getcwd()
SRC = os.path.join(ROOT, "src")
SCRIPTS = os.path.join(ROOT, "scripts")
for p in (SRC, SCRIPTS):
    if p not in sys.path: sys.path.insert(0, p)

# Purge any site-packages micro_lm so we import from src/
for m in list(sys.modules):
    if m == "micro_lm" or m.startswith("micro_lm."):
        del sys.modules[m]

# Legacy pickle compat (old joblib references like 'encoders' / __main__.SBERTEncoder)
def _legacy_install_inline(target="micro_lm.domains.defi.benches.encoders"):
    mod = importlib.import_module(target)
    sys.modules.setdefault("encoders", mod)
    for name in ("SBERTEncoder","SbertEncoder","EmbedVectorizer","SBERTVectorizer"):
        if hasattr(mod, name) and not hasattr(__main__, name):
            setattr(__main__, name, getattr(mod, name))
_legacy_install_inline()

# Use the exact rails-backed helper the CLI uses
from scripts.tier2_benchmark import run_once

# Tier-2 policy: rails + WDD audit; mapper only for scoring if rails abstain
POLICY = {
    "ltv_max": 0.75,
    "hf_min": 1.0,
    "audit": {"backend": "wdd", "mode": "pure", "profile": True, "max_null": 64, "batch": 32},
    "mapper": {"model_path": ".artifacts/defi_mapper.joblib", "confidence_threshold": 0.45},
}

# Minimal but sufficient domain context (register ETH + Aave)
CONTEXT = {
    "oracle": {"age_sec": 5, "max_age_sec": 30},
    "defi": {
        "assets": {
            # add more if needed later (USDC, WETH, etc.)
            "ETH": {"symbol": "ETH", "decimals": 18, "chain": "ethereum"},
        },
        "venues": {
            # name must match what your rails expect (often "aave" or "aave_v3")
            "aave": {"type": "lending", "version": "v3", "chain": "ethereum"},
        },
    },
}

def call(prompt, domain="defi", policy=POLICY, context=CONTEXT, rails="stage11", T=1, show_trace_lines=14):
    out = run_once(domain=domain, prompt=prompt, context=context, policy=policy, rails=rails, T=T)
    print(f"\n=== {prompt!r} ===")
    print("verify:", out.get("verify"))
    print("plan:  ", out.get("plan"))
    print("flags: ", out.get("flags"))
    audit = ((out.get("artifacts") or {}).get("audit") or {})
    trace = audit.get("trace") or audit.get("profile") or audit.get("log")
    if trace:
        lines = trace if isinstance(trace, list) else str(trace).splitlines()
        print("\n[audit trace — first lines]")
        for ln in lines[:show_trace_lines]:
            print(ln)
    return out

# 1) Supported exec: should now produce a plan and verify.ok=True
out1 = call("deposit 10 ETH into aave")
assert (out1.get("plan") or {}).get("sequence"), "expected an executable plan"
assert (out1.get("verify") or {}).get("ok") is True, "expected verify.ok=True for a safe deposit"

# 2) Your earlier SOL/Maker example will still abstain unless you register them:
out2 = call("supply 7.0245 SOL to maker")

# If you want that to pass too, register SOL + maker and try again:
CONTEXT["defi"]["assets"]["SOL"] = {"symbol":"SOL","decimals":9, "chain":"solana"}  # adjust to your rails’ schema
CONTEXT["defi"]["venues"]["maker"] = {"type":"lending", "version":"v3", "chain":"ethereum"}  # or whatever your rails expect
out3 = call("supply 7.0245 SOL to maker")


[debug] mapper fallback for 'deposit 10 ETH into aave' -> deposit_asset

=== 'deposit 10 ETH into aave' ===
verify: {'ok': False, 'reason': 'abstain_non_exec'}
plan:   {}
flags:  {'mapper_fallback': True}


AssertionError: expected an executable plan

In [7]:
import os, sys, warnings, joblib, numpy as np
warnings.filterwarnings("ignore", category=FutureWarning)

# Ensure src/ is importable
SRC = os.path.join(os.getcwd(), "src")
if SRC not in sys.path: sys.path.insert(0, SRC)

# Import Tier-2 runner
from micro_lm.core.runner import run_micro

# Tier-2 style policy = Stage-11 rails + WDD audit + mapper fallback
POLICY = {
    "ltv_max": 0.75,
    "hf_min": 1.0,
    "audit": {"backend": "wdd", "mode": "pure", "profile": False, "max_null": 64, "batch": 32},
    "mapper": {"model_path": ".artifacts/defi_mapper.joblib", "confidence_threshold": 0.45},
}
CONTEXT = {"oracle": {"age_sec": 5, "max_age_sec": 30}}

# Example Tier-2 prompt
prompt = "supply 7.0245 SOL to maker"

# Call the real Stage-11 rails pipeline
out = run_micro(
    domain="defi",
    prompt=prompt,
    context=CONTEXT,
    policy=POLICY,
    rails="stage11",
    T=180,
)

# If no plan was produced, do mapper fallback (like tier2_benchmark)
def _predict_one(model, text):
    if hasattr(model, "predict"): return model.predict([text])[0]
    if hasattr(model, "predict_proba"):
        proba = model.predict_proba([text])[0]
        labels = getattr(model, "classes_", None)
        return labels[int(np.argmax(proba))] if labels is not None else None

seq = (out.get("plan") or {}).get("sequence") or []
top1 = seq[0] if seq else None
if top1 is None and POLICY.get("mapper", {}).get("model_path"):
    mapper = joblib.load(POLICY["mapper"]["model_path"])
    m_top1 = _predict_one(mapper, prompt)
    if m_top1:
        top1 = m_top1
        out.setdefault("flags", {})["mapper_fallback"] = True

print("verify:", out.get("verify"))
print("plan:", out.get("plan"))
print("top1:", top1, "flags:", out.get("flags"))


verify: None
plan: None
top1: deposit_asset flags: {'mapper_fallback': True}
